In [1]:
import os, sys
# project_path = '../'
PWD = os.getenv('PWD')
# os.chdir(project_path)
sys.path.insert(0, PWD)
# sys.path.insert(0, project_path)
print(sys.path)
# os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
import django
django.setup()

['/Users/russmo/Code/mpatlas-dj2/mpatlas', '/Users/russmo/Code/mpatlas-dj2/mpatlas/notebooks', '/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Users/russmo/.venvs/mpa/lib/python3.7/site-packages', '/Users/russmo/.venvs/mpa/lib/python3.7/site-packages/IPython/extensions', '/Users/russmo/.ipython']


In [11]:
from mpa.models import Mpa, mpa_post_save
from wdpa.models import WdpaPoly_new
from django.db.models.signals import post_save, post_delete
from mpa.views import mpas_all_nogeom
import reversion
from reversion.models import Revision
from reversion.models import Version
import datetime
from django.db.models import Q, F, Func, Value, Count
from django.db.models.functions import Concat

from importlib import reload

In [3]:
# Update mpatlas field values before data import
# mpas = mpas_all_nogeom.exclude(
#         Q(country__icontains='MEX') | Q(country__icontains='CHL') | Q(country__icontains='IDN') | 
#         Q(country__icontains='CHN') | Q(country__icontains='JPN') | 
#         Q(sovereign__icontains='MEX') | Q(sovereign__icontains='CHL') | Q(sovereign__icontains='IDN') |
#         Q(sovereign__icontains='CHN') | Q(sovereign__icontains='JPN')
#     ).filter(wdpa_id__isnull=False)
mpas = mpas_all_nogeom.filter(
#     Q(country__icontains='CHL') | Q(sovereign__icontains='CHL')
).filter(wdpa_id__isnull=False)
mpas.count()

20840

In [11]:
mpas.update(
    no_take_mpatlas=F('no_take'),
    no_take_area_mpatlas=F('no_take_area'),
    calc_area_mpatlas=F('calc_area'),
    calc_m_area_mpatlas=F('calc_m_area'),
    status_mpatlas=F('status'),
    status_year_mpatlas=F('status_year'),
)

20840

In [6]:
# Set wdpa_pid for new zones
# If existing MPAtlas sites with this wdpa_id,
# set first non-rejected site to the first PID_A.
# Clear pid for all other MPAtlas sites and flag
# as orphaned zones.

wzones = []
mpasw = [float(wi) for wi in mpas_all_nogeom.filter(wdpa_id__isnull=False).order_by('wdpa_id').values_list('wdpa_id', flat=True)]
polyset = WdpaPoly_new.objects.filter(wdpaid__in=mpasw).only('wdpaid', 'wdpa_pid', 'name', 'desig_eng', 'iso3', 'parent_iso3')
for w in polyset:
    if w.wdpa_pid != str(int(w.wdpaid)):
        wzones.append(w.wdpaid)
wzones = list(set(wzones))
len(wzones)

164

In [9]:
# Assign records with zones to be the first zone.  Do not rerun.
count = 0
for wid in wzones:
    zones = WdpaPoly_new.objects.filter(wdpaid=wid).only('wdpaid', 'wdpa_pid', 'name', 'desig_eng', 'iso3', 'parent_iso3')
    firstpid = zones.order_by('wdpa_pid').first().wdpa_pid
    mpaz = mpas_all_nogeom.filter(wdpa_id=int(wid)).exclude(Q(wdpa_pid__isnull=True) | Q(wdpa_pid__contains='_'))
    # only process mpas that don't yet have wdpa_pid zones, where str(wdpaid) = wdpa_pid,
    # and also don't have null wdpa_pid.  We set some custom zone sites to have null wdpa_pid
    # so they don't get overwritten.  Better to have duplicates to clean up.
    for m in mpaz:
        print(m.mpa_id, m.wdpa_id, m.wdpa_pid)
        m.wdpa_pid = firstpid
        count += 1
        print(count, m.mpa_id, m.wdpa_id, m.wdpa_pid)
        m.save()


14242 555544090 55544090
1 14242 555544090 55544090
ERROR processing mpa 14242:  Cannot resolve keyword 'protection_level' into field. Choices are: access, access_citation, access_info, bbox_lowerleft, bbox_upperright, calc_area, calc_area_mpatlas, calc_m_area, calc_m_area_mpatlas, campaign, candidateinfo, categories, cons_obj_wdpa, conservation_effectiveness, conservation_focus_citation, conservation_focus_info, constancy, constancy_citation, contact, contact_id, country, created_date, datasource, datasource_id, designation, designation_eng, designation_type, eez, eezmembership, fishing, fishing_citation, fishing_info, fishing_protection_details, fishing_protection_level, geog, geom, geom_smerc, glores_status, gov_type, implementation_date, implemented, int_criteria, is_mpa, is_point, iucn_category, long_name, marine, mgmt_auth, mgmt_plan_ref, mgmt_plan_type, modified_date, mpa_id, name, no_take, no_take_area, no_take_area_mpatlas, no_take_area_wdpa, no_take_mpatlas, no_take_wdpa, not

68816877 555637853 555637853
6 68816877 555637853 555637853_A
ERROR processing mpa 68816877:  Cannot resolve keyword 'protection_level' into field. Choices are: access, access_citation, access_info, bbox_lowerleft, bbox_upperright, calc_area, calc_area_mpatlas, calc_m_area, calc_m_area_mpatlas, campaign, candidateinfo, categories, cons_obj_wdpa, conservation_effectiveness, conservation_focus_citation, conservation_focus_info, constancy, constancy_citation, contact, contact_id, country, created_date, datasource, datasource_id, designation, designation_eng, designation_type, eez, eezmembership, fishing, fishing_citation, fishing_info, fishing_protection_details, fishing_protection_level, geog, geom, geom_smerc, glores_status, gov_type, implementation_date, implemented, int_criteria, is_mpa, is_point, iucn_category, long_name, marine, mgmt_auth, mgmt_plan_ref, mgmt_plan_type, modified_date, mpa_id, name, no_take, no_take_area, no_take_area_mpatlas, no_take_area_wdpa, no_take_mpatlas, no_t

68809454 555621668 555621668
11 68809454 555621668 555621668_A
ERROR processing mpa 68809454:  Cannot resolve keyword 'protection_level' into field. Choices are: access, access_citation, access_info, bbox_lowerleft, bbox_upperright, calc_area, calc_area_mpatlas, calc_m_area, calc_m_area_mpatlas, campaign, candidateinfo, categories, cons_obj_wdpa, conservation_effectiveness, conservation_focus_citation, conservation_focus_info, constancy, constancy_citation, contact, contact_id, country, created_date, datasource, datasource_id, designation, designation_eng, designation_type, eez, eezmembership, fishing, fishing_citation, fishing_info, fishing_protection_details, fishing_protection_level, geog, geom, geom_smerc, glores_status, gov_type, implementation_date, implemented, int_criteria, is_mpa, is_point, iucn_category, long_name, marine, mgmt_auth, mgmt_plan_ref, mgmt_plan_type, modified_date, mpa_id, name, no_take, no_take_area, no_take_area_mpatlas, no_take_area_wdpa, no_take_mpatlas, no_

68816568 555599265 555599265
16 68816568 555599265 555599265_A
ERROR processing mpa 68816568:  Cannot resolve keyword 'protection_level' into field. Choices are: access, access_citation, access_info, bbox_lowerleft, bbox_upperright, calc_area, calc_area_mpatlas, calc_m_area, calc_m_area_mpatlas, campaign, candidateinfo, categories, cons_obj_wdpa, conservation_effectiveness, conservation_focus_citation, conservation_focus_info, constancy, constancy_citation, contact, contact_id, country, created_date, datasource, datasource_id, designation, designation_eng, designation_type, eez, eezmembership, fishing, fishing_citation, fishing_info, fishing_protection_details, fishing_protection_level, geog, geom, geom_smerc, glores_status, gov_type, implementation_date, implemented, int_criteria, is_mpa, is_point, iucn_category, long_name, marine, mgmt_auth, mgmt_plan_ref, mgmt_plan_type, modified_date, mpa_id, name, no_take, no_take_area, no_take_area_mpatlas, no_take_area_wdpa, no_take_mpatlas, no_

In [18]:
# If we set more than one site to the same PID, then keep the pid only for the first non-rejected site.
# Set the others to be flagged as orphaned zones.

dups = (
    mpas.filter(wdpa_id__isnull=False, wdpa_pid__contains='_')
    .values('wdpa_pid')
    .annotate(pidcount=Count('wdpa_pid'))
    .values('wdpa_pid')
    .order_by()
    .filter(pidcount__gt=1)
)

duppids = mpas.filter(wdpa_id__isnull=False).filter(wdpa_pid__in=dups)

for pid in dups.values_list('wdpa_pid', flat=True):
    dupmpas = mpas.filter(wdpa_pid=pid).order_by('mpa_id')
    nonrejects = dupmpas.exclude(verification_state='Rejected as MPA').order_by('mpa_id')
    if nonrejects:
        m = nonrejects[0]
    else:
        m = dupmpas[0]
    others = dupmpas.exclude(mpa_id=m.mpa_id)
    for o in others:
        o.categories.add('Orphaned Zone')
    others.update(wdpa_pid=None, notes=Concat(F('notes'), Value(' ORPHANED ZONE, please clean up.')))

In [19]:
# 2020-12-21: Did not run commands below.  I can't remember what they do or if they are even needed.

In [ ]:
from django.db.models import F, Func
mpazz = mpas_all_nogeom.filter(wdpa_id__isnull=False).exclude(wdpa_pid__in=wd)
print(mpazz.count())
mpazz.update(wdpa_pid=Func(F('wdpa_id'), function='CAST', template='%(function)s(%(expressions)s as varchar(52))'))

In [ ]:
mpazx = mpas_all_nogeom.filter(wdpa_id__isnull=False).exclude(wdpa_id__in=[int(w) for w in wzones])
mpazx.update(wdpa_pid=Func(F('wdpa_id'), function='CAST', template='%(function)s(%(expressions)s as varchar(52))'))

In [ ]:
count = 0
mpazy = mpas_all_nogeom.only('wdpa_id', 'mpa_id', 'wdpa_pid').filter(wdpa_id__isnull=False).exclude(wdpa_id__in=[int(w) for w in wzones])
for m in mpazy:
    count += 1
    mywid = m.wdpa_id
    mypid = str(m.wdpa_id)
    myid = m.mpa_id
    m.wdpa_pid = mypid
    print(count, m.mpa_id, m.wdpa_id, m.wdpa_pid)
    #m.save()
    mq = mpas_all_nogeom.filter(mpa_id=myid).update(wdpa_pid=mypid)